In [3]:
import math
from typing import List
from matrix import CSRMatrix as Matrix 
from decimal import Decimal, getcontext

# Сама лаба


## Easy


### Метод гауса


In [4]:
def gauss_solver(A: 'Matrix', b: 'Matrix') -> List['Matrix']:
    n = A.size
    augmented = A.augment(b)  # Расширенная матрица [A|b]
    
    # Прямой ход: приведение к ступенчатому виду
    rank = 0
    pivot_cols = []
    for col in range(augmented.cols - 1):  # Исключаем последний столбец (вектор b)
        # Поиск ненулевого элемента в текущем столбце
        pivot_row = -1
        for row in range(rank, n):
            if abs(augmented[row][col]) > 1e-10:
                pivot_row = row
                break
        if pivot_row == -1:
            continue  # Все элементы столбца нулевые
        
        # Перестановка строк
        augmented.swap_rows(rank, pivot_row)
        # Нормализация ведущей строки
        pivot = augmented[rank][col]
        for c in range(col, augmented.cols):
            augmented[rank][c] /= pivot
        # Исключение элементов ниже ведущего
        for row in range(rank + 1, n):
            factor = augmented[row][col]
            for c in range(col, augmented.cols):
                augmented[row][c] -= factor * augmented[rank][c]
        pivot_cols.append(col)
        rank += 1

    # Проверка на несовместность
    for row in range(rank, n):
        if abs(augmented[row][-1]) > 1e-10:
            raise ValueError("Система несовместна")

    # Определение свободных переменных
    all_vars = list(range(A.cols))
    leading_vars = pivot_cols
    free_vars = [var for var in all_vars if var not in leading_vars]
    
    # Если нет свободных переменных: единственное решение
    if not free_vars:
        x = [0] * A.cols
        for row in reversed(range(rank)):
            x[leading_vars[row]] = augmented[row][-1]
            for col in range(leading_vars[row] + 1, A.cols):
                x[leading_vars[row]] -= augmented[row][col] * x[col]
        return [Matrix([x])]

    # Построение базисных векторов
    basis = []
    for var in free_vars:
        solution = [0] * A.cols
        solution[var] = 1  # Свободная переменная = 1
        # Вычисление главных переменных
        for row in reversed(range(rank)):
            col = leading_vars[row]
            solution[col] = augmented[row][-1]
            for c in range(col + 1, A.cols):
                solution[col] -= augmented[row][c] * solution[c]
        basis.append(Matrix([solution]))
    
    return basis


In [5]:

# A = Matrix([ [1, 2, 3], [4, 5, 6],
#     [7, 8, 9]
# ])
# b = Matrix([[6], [15], [24]])

# solutions = gauss_solver(A, b)  # Возвращает базисные векторы (если система неопределённа)

### Центрирование данных


In [6]:
def center_data(X: 'Matrix') -> 'Matrix':
    """
    Centers the data matrix X by subtracting the mean of each column.

    Input: data matrix X (n x m)
    Output: centered matrix X_centered (n x m)
    """
    n, m = X.size

    if n == 0 or m == 0:
        return type(X)([])

    column_means = [0.0] * m
    x_list = X.to_list()

    for row in x_list:
        for j in range(m):
            column_means[j] += float(row[j]) / n


    mean_matrix_list = [column_means[:] for _ in range(n)]
    mean_matrix = type(X)(mean_matrix_list)

    X_centered = X - mean_matrix

    return X_centered

In [7]:
# Tests
matrix1 = Matrix([[1.0, 2.0],
         [3.0, 4.0],
         [5.0, 6.0]])

centered_matrix1 = center_data(matrix1)
print(centered_matrix1)

matrix2= Matrix([[-1.0, 10.0],
         [ 0.0,  0.0],
         [ 1.0, -10.0]])

centered_matrix2 = center_data(matrix2)
print(centered_matrix2)

matrix3 = Matrix([[1, 2, 3]])
centered_matrix3 = center_data(matrix3)
print(centered_matrix3)

matrix3 = Matrix([[1, 2, 3]]).transpose()
centered_matrix3 = center_data(matrix3)
print(centered_matrix3)


[
 [-2.0 -2.0]
 [ 0.0  0.0]
 [ 2.0  2.0]
]
[
 [-1.0  10.0]
 [ 0.0   0.0]
 [ 1.0 -10.0]
]
[
 [0.0 0.0 0.0]
]
[
 [-1.0]
 [ 0.0]
 [ 1.0]
]


### Вычислить матрицу ковариаций


In [8]:
def covariance_matrix(X_centered: 'Matrix') -> 'Matrix':
    """
    Вход: центрированная матрица X_centered (n×m)
    Выход: матрица ковариаций C (m×m)
    """
    n, m = X_centered.size
    if n <= 1:
        print('z')
        return Matrix([[0] * m] * m)
    return X_centered.transpose() * X_centered * (1/(n-1))

In [9]:
from decimal import Decimal
# --- Tests ---


data1_centered = Matrix([[1, -1], [-1, 1]])
cov_matrix1 = covariance_matrix(data1_centered)
print("Input X_centered:")
print(data1_centered)
print("Computed Covariance Matrix:")
print(cov_matrix1)
expected_cov1 = Matrix([[Decimal(2), Decimal(-2)], [Decimal(-2), Decimal(2)]])
print("-" * 20)


data2_centered = Matrix([[1, 2, 3], [0, 0, 0], [-1, -2, -3]])
cov_matrix2 = covariance_matrix(data2_centered)
print("Input X_centered:")
print(data2_centered)
print("Computed Covariance Matrix:")
print(cov_matrix2)
expected_cov2 = Matrix([[Decimal(1), Decimal(2), Decimal(3)], [Decimal(2), Decimal(4), Decimal(6)], [Decimal(3), Decimal(6), Decimal(9)]])
print("-" * 20)

# Test case 3: Matrix with one sample (should handle this edge case)
print("Test Case 3 (Single Sample):")
data3_centered = Matrix([[10, 20]])
cov_matrix3 = covariance_matrix(data3_centered)
print("Input X_centered:")
print(data3_centered)
print("Computed Covariance Matrix:")
print(cov_matrix3)
expected_cov3 = Matrix([[Decimal(0), Decimal(0)], [Decimal(0), Decimal(0)]])
print("-" * 20)

Input X_centered:
[
 [ 1 -1]
 [-1  1]
]
Computed Covariance Matrix:
[
 [ 2.0 -2.0]
 [-2.0  2.0]
]
--------------------
Input X_centered:
[
 [ 1  2  3]
 [ 0  0  0]
 [-1 -2 -3]
]
Computed Covariance Matrix:
[
 [1.0 2.0 3.0]
 [2.0 4.0 6.0]
 [3.0 6.0 9.0]
]
--------------------
Test Case 3 (Single Sample):
z
Input X_centered:
[
 [10 20]
]
Computed Covariance Matrix:
[
 [0 0]
 [0 0]
]
--------------------


## Normal


### Собственные значения матрицы


In [26]:
def jacobi_step(A: "Matrix", p: int, q: int, angle: float) -> "Matrix":
    """
    Выполняет один шаг метода Якоби (вращение).

    Вход:
    A: текущая матрица
    p, q: индексы внедиагонального элемента
    angle: угол поворота
    Выход: преобразованная матрица A' = J^T * A * J
    """
    n = A.size[0]
    cos_a = math.cos(float(angle))
    sin_a = math.sin(float(angle))
    dtype = A.dtype

    # Construct the rotation matrix J(p, q, angle)
    J_data = [[dtype(0)] * n for _ in range(n)]
    one_val = dtype(1) if dtype is not complex else complex(1)

    for k in range(n):
        J_data[k][k] = one_val

    J_data[p][p] = dtype(cos_a) if dtype is not complex else complex(cos_a)
    J_data[q][q] = dtype(cos_a) if dtype is not complex else complex(cos_a)
    J_data[p][q] = dtype(sin_a) if dtype is not complex else complex(sin_a)
    J_data[q][p] = dtype(-sin_a) if dtype is not complex else complex(-sin_a)

    J = Matrix(J_data)

    # Apply the transformation A' = J^T * A * J
    # Use the built-in transpose method
    J_transpose = J.transpose()
    A_prime = (J_transpose * A) * J

    return A_prime


def jacobi_eigen_decomposition(
    C: "Matrix", tol: float = 1e-6, max_iterations: int = 100
) -> "Matrix":
    """
    Выполняет итерации метода Якоби для приведения симметричной матрицы к диагональному виду.

    Вход:
    C: симметричная матрица (m×m)
    tol: допустимая погрешность (максимальное абсолютное значение внедиагонального элемента)
    max_iterations: максимальное количество итераций
    Выход: матрица, близкая к диагональной (диагональные элементы - собственные значения)
    """
    if C.size[0] != C.size[1]:
        raise ValueError("Input matrix must be square.")
    if C.dtype is complex:
        raise TypeError("Jacobi method implementation expects a real matrix.")

    n = C.size[0]
    A = Matrix(C.to_list())  # Work on a copy

    for iteration in range(max_iterations):
        # Find the largest off-diagonal element (in absolute value)
        max_val = 0
        p, q = 0, 0
        for i in range(n):
            for j in range(i + 1, n):  # Only check upper triangle
                abs_val = abs(A[i, j])
                if abs_val > max_val:
                    max_val = abs_val
                    p, q = i, j

        # Check for convergence based on the largest off-diagonal element
        if max_val < tol:
            break

        # Calculate rotation angle
        app = A[p, p]
        aqq = A[q, q]
        apq = A[p, q]

        # Calculate tan(2*theta) and then theta
        # More stable calculation for theta using atan2
        # theta = 0.5 * atan2(2*a_pq, a_pp - a_qq)
        # Handle potential division by zero or very small difference
        denominator = app - aqq
        if abs(denominator) < 1e-12:  # Use a small epsilon for comparison
            # Angle is pi/4 or -pi/4 depending on the sign of apq
            angle = math.pi / 4.0 if apq >= 0 else -math.pi / 4.0
        else:
            angle = math.atan2(2.0 * float(apq), float(denominator)) / 2.0

        # Apply the rotation step
        A = jacobi_step(A, p, q, angle)

        # Optional: Ensure symmetry after rotation (due to floating point errors)
        # for i in range(n):
        #     for j in range(i + 1, n):
        #         avg = (A[i, j] + A[j, i]) / A.dtype(2)
        #         A._mat[i][j] = avg
        #         A._mat[j][i] = avg

    return A


def find_eigenvalues(
    C: "Matrix", tol: float = 1e-6, max_iterations: int = 100
) -> list[float]:
    """
    Находит собственные значения симметричной матрицы методом Якоби.

    Вход:
    C: симметричная матрица ковариаций (m×m)
    tol: допустимая погрешность (максимальное абсолютное значение внедиагонального элемента)
    max_iterations: максимальное количество итераций
    Выход: список вещественных собственных значений
    """
    # Perform the Jacobi decomposition
    diagonalized_matrix = jacobi_eigen_decomposition(C, tol, max_iterations)

    # Extract eigenvalues from the diagonal
    n = diagonalized_matrix.size[0]
    eigenvalues = [float(diagonalized_matrix[i, i]) for i in range(n)]

    # Sort eigenvalues in descending order
    eigenvalues.sort(reverse=True)

    return eigenvalues



In [27]:
# --- Тест функции ---
print("Тестируем функцию find_eigenvalues:")

# Пример симметричной матрицы 2x2 с известными собственными значениями (3 и 1)
# Matrix: [[2, 1], [1, 2]]
matrix_data_2x2 = [
    [2.0, 1.0],
    [1.0, 2.0],
]  # Use float for consistency with math functions
mat_2x2 = Matrix(matrix_data_2x2)

eigenvalues_2x2 = find_eigenvalues(mat_2x2)
print(f"Матрица:\n{mat_2x2}")
print(f"Найденные собственные значения: {eigenvalues_2x2}")
# Ожидаемые собственные значения: примерно [3.0, 1.0]

print("\n")

# Пример симметричной матрицы 3x3 (ковариационная матрица может быть такого размера)
# Matrix: [[4, 1, 2], [1, 5, 0], [2, 0, 3]]
# Eigenvalues are approx 6.44, 3.10, 2.45
matrix_data_3x3 = [[4.0, 1.0, 2.0], [1.0, 5.0, 0.0], [2.0, 0.0, 3.0]]
mat_3x3 = Matrix(matrix_data_3x3)

eigenvalues_3x3 = find_eigenvalues(mat_3x3)
print(f"Матрица:\n{mat_3x3}")
print(f"Найденные собственные значения: {eigenvalues_3x3}")
# Ожидаемые собственные значения: примерно в районе [6.44, 3.10, 2.45]


Тестируем функцию find_eigenvalues:
Матрица:
[
 [2.0 1.0]
 [1.0 2.0]
]
Найденные собственные значения: [3.0, 1.0]


Матрица:
[
 [4.0 1.0 2.0]
 [1.0 5.0 0.0]
 [2.0 0.0 3.0]
]
Найденные собственные значения: [5.804112903938886, 4.86274801912311, 1.3331390769379987]


### Собственные векторы матрицы


In [12]:
def find_eigenvectors(C: 'Matrix', eigenvalues: List[float]) -> List['Matrix']:
    """
    Вход:
    C: матрица ковариаций (m×m)
    eigenvalues: список собственных значений
    Выход: список собственных векторов (каждый вектор - объект Matrix)
    """
    pass

In [13]:
# для тестов

### доля объясненной дисперсии


In [14]:
def explained_variance_ratio(eigenvalues: List[float], k: int) -> float:
    """
    Вход:
    eigenvalues: список собственных значений
    k: число компонент
    Выход: доля объяснённой дисперсии
    """
    pass

In [15]:
# для тестов

## Hard


### полный алгоритм PCA


In [16]:
'''1. Центрирование данных.
2. Вычисление матрицы выборочных ковариаций.
3. Нахождение собственных значений и векторов.
4. Проекция данных на главные компоненты.'''
def pca(X: 'Matrix', k: int) -> tuple['Matrix', float]:
    """
    Вход:
    X: матрица данных (n×m)
    k: число главных компонент
    Выход:
    X_proj: проекция данных (n×k)
    : доля объяснённой дисперсии
    """
    pass

In [17]:
# для тестов

In [18]:
# для тестов

### Проекция данных на две главные компоненты


In [19]:
# Для импортов 

In [20]:
def plot_pca_projection(X_proj: 'Matrix') -> Figure:
    """
    Вход: проекция данных X_proj (n×2)
    Выход: объект Figure из Matplotlib
    """
    pass

NameError: name 'Figure' is not defined

In [ ]:
# для тестов

### MSE


In [ ]:
def reconstruction_error(X_orig: 'Matrix', X_recon: 'Matrix') -> float:
    """
    Вход:
    X_orig: исходные данные (n×m)
    X_recon: восстановленные данные (n×m)
    Выход: среднеквадратическая ошибка MSE
    """
    pass

In [ ]:
# для тестов

## Expert


### Автоматический выбор числа главных компонент на основе порога объяснённой дисперсии


In [ ]:
def auto_select_k(eigenvalues: List[float], threshold: float = 0.95) -> int:
    """
    Вход:
    eigenvalues: список собственных значений
    threshold: порог объяснённой дисперсии
    Выход: оптимальное число главных компонент k
    """
    pass

In [ ]:
# Для тестов и результатов

### Обработать пропущенные значения в данных


In [ ]:
def handle_missing_values(X: 'Matrix') -> 'Matrix':
    """
    Вход: матрица данных X (n×m) с возможными NaN
    Выход: матрица данных X_filled (n×m) без NaN
    """
    pass

In [ ]:
# Для тестов и результатов

### Влияние шума на PCA


In [ ]:
def add_noise_and_compare(X: 'Matrix', noise_level: float = 0.1):
    """
    Вход:
    X: матрица данных (n×m)
    noise_level: уровень шума (доля от стандартного отклонения)
    Выход: результаты PCA до и после добавления шума.
    В этом задании можете проявить творческие способности, поэтому выходные данные не
    ,→ типизированы.
    """
    pass

In [ ]:
# Для тестов и результатов работы

### PCA к реальному датасету


In [ ]:
# Для импорттов

In [ ]:
# Для загрузки

In [ ]:
def apply_pca_to_dataset(dataset_name: str, k: int) -> Tuple['Matrix', float]:
    """
    Вход:
    dataset_name: название датасета
    k: число главных компонент
    Выход: кортеж (проекция данных, качество модели)
    """
    pass

In [ ]:
# Применение и результаты